In [ ]:
from __future__ import division
import numpy as np
from PIL import Image
from scipy import misc
from skimage import data
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
%matplotlib inline
import requests
from StringIO import StringIO
import skimage.measure
import math
import pickle
import csv

def unpickle(file):
    import cPickle
    with open(file, 'rb') as fo:
        dict = cPickle.load(fo)
    return dict


def squash_pixels(value):
    if value<0:
        return 0
    elif value<255:
        return value
    else:
        return 255

def conv_2d_kernel(img_array, kernel, squash_pixel=True):
    padded_array=np.pad(img_array, (1,1), 'constant')
    kernel_width = kernel.shape[0]
    kernel_height = kernel.shape[1]
    
    transformed_array = np.zeros(img_array.shape)
    
    for i in xrange(padded_array.shape[0] - kernel_width +1):
        for j in xrange(padded_array.shape[1] - kernel_height + 1):
            temp_array = padded_array[i:i+kernel_width, j:j+kernel_height]
            if squash_pixel:
                transformed_array[i,j] = squash_pixels(np.sum(temp_array*kernel))
            else:
                transformed_array[i,j] = np.sum(temp_array*kernel)                
    return transformed_array
                
                    
def relu_layer(x):
        #turn all negative values in a matrix into zeros
        z = np.zeros_like(x)
        return np.where(x>z,x,z)
    
    
def max_pooling(img_array, pool_size):
    img_width = img_array.shape[0]
    img_height = img_array.shape[1]
    
    res_array = skimage.measure.block_reduce(img_array, (pool_size,pool_size), np.max)
    return res_array
#print img

def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_derivative(x):
    return x*(1-x)


    
input_layer_neurons = 16
output_layer_neurons = 1        
hidden_layer_neurons = 8
w1 = np.random.uniform(size=(16, 8))
b1 = np.random.uniform(size=(1, 8))
w2 = np.random.uniform(size=(8,1))
b2 = np.random.uniform(size=(1, 1))


def fcl_for_training(dataset, epoch,y,lr):
    global w1
    global b1
    global w2
    global b2
    for i in range(epoch):
        #FEED FORWARD
        hidden_layer_input = np.dot(dataset,w1) + b1
        hidden_layer_activations = sigmoid(hidden_layer_input)
        output_layer_input = np.dot(hidden_layer_activations, w2) + b2
        output = sigmoid(output_layer_input)



        #BACKPROPAGATION
        E = y-output
        slope_output_layer = sigmoid_derivative(output)
        slope_hidden_layer = sigmoid_derivative(hidden_layer_activations)
        d_output = E * slope_output_layer
        error_at_hidden_layer = d_output.dot(w2.T)
        d_hidden_layer = error_at_hidden_layer*slope_hidden_layer
        w2 += hidden_layer_activations.T.dot(d_output)*lr
        b2 += np.sum(d_output, axis=0, keepdims=True) * lr
        w1 += dataset.T.dot(d_hidden_layer) * lr
        b1 += np.sum(d_hidden_layer, axis=0, keepdims=True) * lr

        
        
        
        
def fcl_for_testing(dataset, epoch):
    global w1
    global b1
    global w2
    global b2
    for i in range(epoch):                    
        hidden_layer_input = np.dot(dataset,w1) + b1
        hidden_layer_activations = sigmoid(hidden_layer_input)
        output_layer_input = np.dot(hidden_layer_activations, w2) + b2
        output = sigmoid(output_layer_input)
        
    return output
            
    
#img = unpickle('cifar-10-batches-py/data_batch_1')['data']
#labels = unpickle("cifar-10-batches-py/data_batch_1")['labels']

kernel1 = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
X = []
for i in range(1,50000):
    img = np.array(Image.open("train/%d.png" %i))
    img = img[:,:,0]
    X.append(img)

epoch = 5000
lr = 0.1


#label_names =  pickle.load(open("cifar-10-batches-py/batches.meta", 'rb'))['label_names']
#print label_names


file_to_read_from = 'trainLabels.csv'

#initializing as many lists as the columns you want (not all)
col1, col2, col3 = [], [], []
with open(file_to_read_from, 'r') as file_in:
    reader = csv.reader(file_in, delimiter=',') #might as well be ',', '\t' etc
    for row in reader:
        col1.append(row[0])
        col2.append(row[1])

map_dict = {
    "frog": np.array([[1],[0],[0], [0], [0],[0],[0], [0], [0],[0],[0], [0], [0],[0], [0], [0]])  ,
    "airplane": np.array([[0],[1],[0], [0], [0],[0],[0], [0], [0],[0],[0], [0], [0],[0], [0], [0]]),
    "automobile": np.array([[0],[0],[1], [0], [0],[0],[0], [0], [0],[0],[0], [0], [0],[0], [0], [0]]),
    "truck": np.array([[0],[0],[0], [1], [0],[0],[0], [0], [0],[0],[0], [0], [0],[0], [0], [0]]),
    "deer": np.array([[0],[0],[0], [0], [1],[0],[0], [0], [0],[0],[0], [0], [0],[0], [0], [0]]),
    "bird": np.array([[0],[0],[0], [0], [0],[1],[0], [0], [0],[0],[0], [0], [0],[0], [0], [0]]),
    "cat": np.array([[0],[0],[0], [0], [0],[0],[1], [0], [0],[0],[0], [0], [0],[0], [0], [0]]),
    "dog": np.array([[0],[0],[0], [0], [0],[0],[0], [1], [0],[0],[0], [0], [0],[0], [0], [0]]),
    "horse": np.array([[0],[0],[0], [0], [0],[0],[0], [0], [1],[0],[0], [0], [0],[0], [0], [0]]),
    "ship": np.array([[0],[0],[0], [0], [0],[0],[0], [0], [0],[1],[0], [0], [0],[0], [0], [0]])
}


i=0

for img_arr in X:
    im1 = conv_2d_kernel(img_arr, kernel1)
    im2 = relu_layer(im1)
    im3 = max_pooling(im2, 2)
    fin_op = fcl_for_training(im3, epoch, map_dict[col2[i]], lr)
    i=i+1


img_arra = np.array(Image.open("deer.png"))
img_arra = img_array[:,:,0]
ime1 = conv_2d_kernel(img_arra, kernel1)
ime2 = relu_layer(ime1)
ime3 = max_pooling(im2, 2)
print fcl_for_testing(ime3, epoch)


    
#y1=conv_2d_kernel(img, kernel1)
#y1=relu_layer(y1)
#y=max_pooling(y1,2)
#fcl(X, epoch,ao,lr)


'''
f,ax_array = plt.subplots(3)
f.set_figheight(10)
f.set_figwidth(15)
ax_array[0].imshow(img, cmap = plt.get_cmap('gray'))
ax_array[1].imshow(y1, cmap = plt.get_cmap('gray'))
ax_array[2].imshow(y, cmap = plt.get_cmap('gray'))
'''






        




/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:63: RuntimeWarning: overflow encountered in exp
